In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from bs4 import BeautifulSoup
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud





df = pd.read_json("catalog.json")
#df.head()
tdf = df.transpose()
tdf.head()



,crse,description,name,subj
ARCH-1200,1200,Two-week introduction to architecture for risi...,Beginners Architecture Career Discovery Program,ARCH
ARCH-1210,1210,Similar structure as the Beginners program. Th...,Advanced Architecture Career Discovery Program,ARCH
ARCH-2150,2150,This course will examine the spectrum of archi...,The Ethos of Architecture,ARCH
ARCH-2160,2160,This course continues the inquiry begun in ARC...,Architectural Media,ARCH
ARCH-2330,2330,Introduction to Structures introduces the stud...,Structures 1,ARCH


In [2]:
'''

--------------------- DATA CLEANING -----------------------

'''
import contractions # expands contractions (ex: I'll => I will)
'''
FUNCTIONS
'''

def contraction_expand(df):
    return contractions.fix(df)

def lowercase_text(df):
    return df.lower()

def string_text(df):
    return str(df)

def remove_user(df):
    return re.sub('@[^\s]+','', df)

def remove_link(df):
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , df)

def remove_email(df):
    return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", df)

def remove_time(df):
    df = re.sub(r'\d+:\d{2}', "", df)
    df = re.sub(r' am.', "", df)
    return re.sub(r' pm.', "", df)

def remove_symbols(df):
    df = re.sub(r'\\+', " ", df)
    df = re.sub(r'\,+', " ", df)
    return re.sub(r'\"+', " ", df)

def remove_short_words(df):
    df = re.sub(r'\b\w{1,3}\b', "", df)
    return df
    

def strip_text(df):
    return df.strip()


'''
DRIVER FUNCTION
'''

def clean_data(df):
    df = string_text(df)
    df = contraction_expand(df)
    df = lowercase_text(df)
    df = remove_email(df)
    df = remove_link(df)
    df = remove_time(df)
    df = remove_user(df)
    df = strip_text(df)
    df = remove_symbols(df)
    return df

# Seperate Function to handle removing special characters
def remove_special_chars(df):
    df = re.sub(r'[^\w]+', " ", df)
    df = ' '.join(df.split())
    return df

def remove_digits(df):
    df = re.sub(r'\d+', '', df)
    return df

In [3]:
tdf['description'] = tdf['description'].apply(clean_data)


In [4]:
# python module based of Numpy and Scipy for NLP preprocessing
from gensim.parsing.preprocessing import STOPWORDS
# handy function for removing stopwords
# simply have to apply this function to the dataset
from gensim.parsing.preprocessing import remove_stopwords

In [5]:
tdf['description'] = tdf['description'].apply(remove_stopwords)
tdf['description'] = tdf['description'].apply(remove_digits)
tdf['description'] = tdf['description'].apply(remove_special_chars)
tdf['description'] = tdf['description'].apply(remove_short_words)


'''
# write cleaned df to json
clean = tdf.transpose()
clean.to_json("out.json", orient='columns')
clean.head()
# write descriptions to json
tdf['description'].to_json("clean_desc.json", orient='columns')
'''

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

req = "Recursion, Binary Trees, Hash Tables, Multiple Inheritance & Exception..."

# get TFIDF vectorizer b/w request and each course description
# use cosine similarity to get pairwise similarities b/w all samples in the data
# add to list, and get the max cosine similarity

vals = {}
for i in range(len(tdf['description'])):
    tfidf = TfidfVectorizer()
    data = [req, tdf['description'][i]]
    mat = tfidf.fit_transform(data)
    sim_mat = cosine_similarity(mat)
    vals.append(sim_mat[0][1])

print(max(vals))    

0.07031246299022464
